In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Set up ##

# Download data file
home_data = pd.read_csv('../input/home-data-for-ml-course/train.csv', index_col='Id')

# Clean DataFrame of targets with no value
home_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

# Clean DataFrame of columns with missing values
cols_with_missing = [col for col in home_data.columns if home_data[col].isnull().any()] 
home_data.drop(cols_with_missing, axis=1, inplace=True)

# Define the prediction target
y = home_data.SalePrice

# Define the features
X = home_data.drop(['SalePrice'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Select categorical columns with relatively low cardinality
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# ('ordinal', OrdinalEncoder(handle_unknown='error')),

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

## Model data ##

# Define
my_model = XGBRegressor(n_estimators=5000, learning_rate=0.025)
#                         early_stopping_rounds=5, early_stopping_metric='mae')
my_pipeline = Pipeline(steps=[
                    ('preprocessor', preprocessor),
                    ('model', my_model)])

# Fit
my_pipeline.fit(X_train, y_train,)
#                 eval_set=[(X_valid, y_valid)])

# Predict
preds = my_pipeline.predict(X_valid)

# Evaluate
mae = mean_absolute_error(y_valid, preds)
print('\n'+str(mae))

## Create competition submission ##

X_test = pd.read_csv('../input/home-data-for-ml-course/test.csv', index_col='Id')

# Preprocess test data
final_X_test = X_test[my_cols].copy()

# Get test predictions
preds_test = my_pipeline.predict(final_X_test)

# Save test predictions to file
output = pd.DataFrame({'Id': final_X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv

16266.608438570205
